In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load data
data_test = np.load('channel_test.npy')
data_train = np.load('channel_train.npy')

# Define OFDM Parameters
N = 64          # Number of subcarriers
CP = 16         # Length of cyclic prefix
P = 8           # Number of pilot carriers per symbol
pilot_value = 3 + 3j # The value of the pilot symbols

# Indices of pilot carriers in frequency domain
all_carriers = np.arange(N)
pilot_carriers = all_carriers[::N//P]
data_carriers = np.delete(all_carriers, pilot_carriers)

# Function to generate an OFDM symbol
def generate_ofdm_symbol(data_bits, pilot_value):
    data_symbols = 2 * data_bits - 1  # BPSK modulation
    ofdm_symbol = np.zeros(N, dtype=complex)
    ofdm_symbol[pilot_carriers] = pilot_value
    ofdm_symbol[data_carriers] = data_symbols
    ifft_data = np.fft.ifft(ofdm_symbol)
    ofdm_with_cp = np.hstack([ifft_data[-CP:], ifft_data])
    return ofdm_with_cp, data_symbols

# Function to simulate channel and add noise
def channel_with_noise(ofdm_with_cp, h, SNR_dB):
    channel_output = np.convolve(ofdm_with_cp, h, mode='same')
    noise_power = 10**(-SNR_dB/10)
    noise = np.sqrt(noise_power/2) * (np.random.randn(len(channel_output)) + 1j * np.random.randn(len(channel_output)))
    rx_signal = channel_output + noise
    return rx_signal

# LS Receiver Function
def ls_receiver(rx_signal, pilot_value):
    rx_signal_no_cp = rx_signal[CP:(CP + N)]
    rx_fft = np.fft.fft(rx_signal_no_cp)
    H_est = rx_fft[pilot_carriers] / pilot_value
    H_interpolated = np.interp(all_carriers, pilot_carriers, H_est)
    rx_data_symbols = rx_fft / H_interpolated
    return rx_data_symbols

# Bit Error Rate Calculation
def bit_err(data_bits, rx_data_symbols):
    rx_data_bits = np.real(rx_data_symbols[data_carriers]) > 0
    bit_errors = np.sum(data_bits != rx_data_bits)
    ber = bit_errors / len(data_bits)
    return ber

# Channel impulse response
h = np.array([0.9 + 0.1j, 0.5 + 0.2j])  # Example channel impulse response

# Simulation
BER = []
for SNR_dB in range(5, 30, 5):
    ber_list = []
    for _ in range(1000):  # Number of OFDM symbols
        data_bits = np.random.binomial(n=1, p=0.5, size=(len(data_carriers),))
        ofdm_with_cp, _ = generate_ofdm_symbol(data_bits, pilot_value)
        rx_signal = channel_with_noise(ofdm_with_cp, h, SNR_dB)
        rx_data_symbols = ls_receiver(rx_signal, pilot_value)
        ber = bit_err(data_bits, rx_data_symbols)
        ber_list.append(ber)
    BER.append(np.mean(ber_list))
    print(f"SNR: {SNR_dB} dB, BER: {BER[-1]}")

# Plotting the results
plt.plot(range(5, 30, 5), BER)
plt.xlabel('SNR (dB)')
plt.ylabel('BER')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with LS Receiver')
plt.grid('on')
plt.show()

In [ ]:
def plot_tanh_relu_mish():
    # Generate values for x
    x = np.linspace(-10, 10, 100)

    # Compute tanh values for corresponding x
    tanh = np.tanh(x)

    

    # Compute ReLU values for corresponding x
    relu = np.maximum(0, x)


    # Compute values for corresponding x
    mish = x*np.tanh(np.log(1+math.e**x))
    # Plot the function
    '''
    plt.plot(x, mish)
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.title('Mish Activation Function')
    plt.grid(True)
    plt.show()   
    '''
    fig, axs = plt.subplots(1, 3,figsize=(30,10))
    axs[ 0].plot(x, tanh)
    axs[ 0].set_title('Tanh Activation Function')
    axs[0].set_xlabel('Input')
    axs[0].set_ylabel('Output')
    axs[ 1].plot(x, relu, 'tab:orange')
    axs[ 1].set_title('ReLU Activation Function')
    axs[1].set_xlabel('Input')
    axs[1].set_ylabel('Output')
    axs[ 2].plot(x, mish, 'tab:red')
    axs[ 2].set_title('Mish Activation Function')
    axs[2].set_xlabel('Input')
    axs[2].set_ylabel('Output')
    axs[0].grid('on');axs[1].grid('on');axs[2].grid('on');


In [ ]:
plot_tanh_relu_mish()